## RAG (Retrieval Augmented Generation) Pattern
This example demonstrates a Retrieval-Augmented Generation (RAG) pattern using pgVectorStore and LangChain to implement an Aircraft Maintenance Assistance application. It leverages external datasets such as maintenance.csv and aircraft_taxonomy.csv, and uses a privately hosted OpenAI-compatible embedding model to generate embeddings for both queries and text. The chat model retrieves the top three most relevant results from the pgVectorStore embeddings. 

### Step 0: Set the working directory for the project

In [21]:
# set the working directory for the project
%cd /home/vcap/app/cf-jupyterlab-workshop

/home/vcap/app/cf-jupyterlab-workshop


### Step 1: Import the dependencies

In [22]:
## Ingestion pipeline to load data
## Install following Modules - langchain-postgres, psycopg2
import os
import json
import pandas as pd
import requests
import httpx
from sqlalchemy import create_engine, text
from langchain.docstore.document import Document
from langchain_postgres.vectorstores import PGVector
from cfenv import AppEnv
import sys, os
import warnings
from tanzu_utils import CFGenAIService
warnings.filterwarnings('ignore')

### Step 2: Set up the OpenAI compatible embedding model credentials

In [23]:
# -----------------------------
# Load services from env
# -----------------------------
env = AppEnv()
# -----------------------------
# Embedding service details
# -----------------------------
embedding_service = CFGenAIService("tanzu-nomic-embed-text")

# List available models
embedding_models = embedding_service.list_models()
for m in embedding_models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")


api_base = embedding_service.api_base + "/openai/v1"
api_key = embedding_service.api_key
model_name = embedding_models[0]["name"]

print("Embedding model:", model_name)

- nomic-embed-text-v1025 (capabilities: EMBEDDING)
Embedding model: nomic-embed-text-v1025


### Step 3: Connect to pgvector store and test the connection

In [24]:
# -----------------------------
# Database connection
# -----------------------------
db_service = env.get_service(name="vector-db")
db_credentials = db_service.credentials
db_uri = db_credentials["uri"]

engine = create_engine(db_uri)

# Test DB connection
with engine.connect() as conn:
    version = conn.execute(text("SELECT version();")).fetchone()
    print("Connected to:", version[0])

Connected to: PostgreSQL 16.6 (VMware Postgres 16.6.0) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit


### Step 4: Initialize a vector store

In [25]:
# -----------------------------
# Embedding function (REST call)
# -----------------------------
url = api_base + "/embeddings"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

def embed_text(text: str):
    payload = {"model": model_name, "input": text}
    resp = requests.post(url, headers=headers, json=payload, verify=False)
    resp.raise_for_status()
    return resp.json()["data"][0]["embedding"]

class CustomEmbeddings:
    def embed_documents(self, texts): return [embed_text(t) for t in texts]
    def embed_query(self, text): return embed_text(text)

embedding = CustomEmbeddings()

# -----------------------------
# PGVector setup
# -----------------------------
vectorstore = PGVector(
    embeddings=embedding,
    connection=db_uri,
    collection_name="maintenance_and_taxonomy",
    use_jsonb=True
)

### Step 5: Set up OpenAI compatible chat model credentials and initialize a chat model

In [27]:
import os
import requests
import json
import httpx
from openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from langchain_openai import OpenAIEmbeddings
from datetime import date
import warnings
import ssl
from langchain_community.embeddings import OllamaEmbeddings
from openai import OpenAI
from langchain.chains import RetrievalQA

# Optional: configure custom http client
httpx_client = httpx.Client(http2=True, verify=False, timeout=30.0)
# -----------------------------
# cat service details
# -----------------------------
chat_service = CFGenAIService("tanzu-gpt-oss-120b")

# List available models
chat_models = chat_service.list_models()
for m in chat_models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")

chat_api_base = chat_service.api_base + "/openai/v1"
chat_api_key = chat_service.api_key
chat_model_name = chat_models[0]["name"]

# Initialize LLM with credentials from cfenv
llm = ChatOpenAI(
    temperature=0.9,
    model=chat_model_name,
    base_url=chat_api_base,
    api_key=chat_api_key,
    http_client=httpx_client
)

- openai/gpt-oss-120b (capabilities: CHAT, TOOLS)


### Step 6: Create a retriever from your vectorstore and execute a query

In [28]:
# Create a retriever from your vectorstore
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})

# Build a RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

# Ask a question
query = "Which aircraft equipment has reported issues with hydraulic leaks?"
result = qa.run(query)
print(result)

The reported hydraulic‑leak issue is with **Landing Gear B**.
